Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [111]:
from random import choices

import lab9_lib

# Local Search:
Local search is a heuristic method for solving computationally hard optimization problems. Local search can be used on problems that can be formulated as finding a solution maximizing a criterion among a number of candidate solutions. Local search algorithms move from solution to solution in the space of candidate solutions (the search space) by applying local changes, until a solution deemed optimal is found or a time bound is elapsed.

## Evolutionary Algorithm implementation :


Voglio trovare la combinazione di 0/1 che massimizza la fitness minimizzando il numero di chiamate alla fitness.
Il prof prende combinazioni a caso di 0 e 1 in una stringa da 50 bit e la valuta chiamando la fitness, che, ogni volta che é chiamata, incrementa un contatore.

Step One: 
  - Generate the initial population of individuals randomly. (First generation)

Step Two: 
  - Repeat the following regenerational steps until termination:
    - Evaluate the fitness of each individual in the population (time limit, sufficient fitness achieved, etc.)
    - Select the fittest individuals for reproduction. (Parents)
    - Breed new individuals through crossover and mutation operations to give birth to offspring.
    - Replace the least-fit individuals of the population with new individuals.

In [112]:
µ = 5
λ = 5 * µ 
o = 1

In [121]:
def init_population():
    return [choices([0, 1], k=1000) for _ in range(µ)]

def evaluate_population(population, fitness):
    return [[individual, fitness(individual), o] for individual in population]

def select_parents(offspring, Best):
    offspring = sorted(offspring, key=lambda i : i[1], reverse=True)
    if Best is None or offspring[-1][1] > Best[1]:
        Best = offspring[-1]
        print(f"Best weights: {Best[0]}, fitness: {Best[1]}, sigma: {Best[2]}")
    return offspring[-µ :]

In [122]:
Best = None
fitness = lab9_lib.make_problem(1)
population = init_population()
population = evaluate_population(population, fitness)
print([i[1] for i in population])
p = select_parents(population, Best)
print([i[1] for i in p])

[0.51, 0.507, 0.488, 0.514, 0.486]
Best weights: [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,

In [115]:
def Evolutionary_algorithm(fitness):
    Best = None
    # 1. Initialize population
    population = init_population()
    # 2. Evaluate population
    population = evaluate_population(population, fitness)
    # 3. Repeat
    for i in range(100):
        # 3.1 Select parents
        parents = select_parents(population, Best)
        # 3.2 Crossover
        offspring = crossover(parents)
        # 3.3 Mutate
        offspring = mutate(offspring)
        # 3.4 Evaluate offspring
        offspring = evaluate_population(offspring)
        # 3.5 Select survivors
        population = select_survivors(population, offspring)
    # 4. Return best individual
    return # best_individual(population)

In [116]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

01110011011110001100001001010111001000000101001010: 44.00%
11001100010000011000111100110110001111000000000110: 42.00%
11110110000110100110000011001111001100010100011101: 50.00%
10101111001010001001001010011011101010010111010001: 50.00%
01100101011101100111011011111110100010011110111110: 64.00%
00011000100011110111001110010010100110001100000000: 40.00%
10101100010110100011011011110111001101001000010101: 52.00%
10001110101110001100101001111100110010000111000010: 48.00%
00001001100100000010101100001101110011000001010010: 36.00%
10101110111000100000001101110000000011110101110010: 46.00%
10


In [117]:
fitness = lab9_lib.make_problem(2)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11100100100101010010101101001000101001111001101000: 24.00%
10100001100100000010100001110110100101111010111111: 25.80%
10111111100110110010101110011001111010010010100000: 32.00%
11110100111001100010100100001100111100010110100011: 23.60%
11110000101010100000111111101111110100011010000001: 27.80%
00110000011001111000011111100101010110010111010100: 30.20%
01011001100001010011111101111011100100001111111111: 31.20%
00100110010011000001100010110101010100011001110110: 24.20%
11011010001000001101110010101110101000110011000110: 28.20%
10010010001001110101111000010010110011011001001011: 48.00%
10


In [118]:
fitness = lab9_lib.make_problem(5)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10000011100101011011011101110101000001111000000111: 22.91%
11110101110011001101101110100110010101000000110111: 12.67%
11000101111111101000101111100110000010000111010110: 12.69%
01011100101011011111000100010011111110000101111110: 14.67%
10101000110111000011010010001000101001011110100000: 12.91%
01000101000001010111100000110111110001101111011011: 34.94%
10011010111001000101011001000000011000100010110001: 10.91%
11000110110100011000111100000000001010111001010010: 22.94%
11100100101111010111001101110000001101101110010011: 14.47%
10001011110100011010010010101011110110010010111110: 14.89%
10


In [119]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00011000111001011001011100010010110100011010111010: 7.33%
01001110100000101110111110111111001001010011101101: 9.11%
00111100001111000000001100101001100101001101100001: 15.36%
11010010011010110010100000001000111110010010111011: 41.58%
11010001001110100111010110101110110010001010110001: 7.33%
10101001010111010001101010101000001010010011011001: 15.34%
11100111001110100100111110110101110011010110100110: 29.33%
00011101111110010111011111110110101010101110000001: 15.33%
01111100110011000010010111101101110110011110111111: 47.34%
01010110101101001001110100001101100100011000100000: 7.33%
10
